Parte 1 XPath - Acesse o arquivo XML em anexo

Exercício 1:

Utilize XPath absoluto com a biblioteca lxml para selecionar:

    O nome do diretor do segundo filme listado;
    O id do penúltimo filme listado, independentemente da quantidade de filmes na lista. 

In [2]:
import lxml
from lxml import etree
from lxml.cssselect import CSSSelector
from lxml import html
import lxml.html

In [3]:


# Parse XML
tree = lxml.etree.parse('..\data\TP3.xml')

# Diretor do segundo filme listado
diretor_segundo_filme = tree.xpath('/library/movies/movie[2]/director/text()')
print(diretor_segundo_filme)

# ID do penúltimo filme
penultimo_filme_id = tree.xpath('/library/movies/movie[last()-1]/@id')
print(penultimo_filme_id)

['Francis Ford Coppola']
['102']


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gabri\AppData\Local\Temp\ipykernel_12048\3271416557.py:2: SyntaxWarning: invalid escape sequence '\d'
  tree = lxml.etree.parse('..\data\TP3.xml')


Exercício 2:

Utilize XPath relativo com a biblioteca lxml para selecionar:

    Os títulos dos livros ou filmes da categoria fantasia;
    Os nomes (siglas) das moedas utilizadas nos preços que não são dólares americanos (USD);
    Os preços com valor numérico maior que 15 (independentemente da moeda);
    Todos os títulos que começam com "The".


In [4]:
# Títulos da categoria fantasia
titulos_fantasia = tree.xpath('//book[@category="fantasy"]/title/text()')
print(titulos_fantasia)

# Moedas que não são USD
moedas_nao_usd = tree.xpath('//price[@currency!="USD"]/@currency')
print(moedas_nao_usd)

# Preços com valor numérico maior que 15
precos_maiores_que_15 = tree.xpath('//price[. > 15]/text()')
print(precos_maiores_que_15)

# Títulos que começam com "The"
titulos_com_the = tree.xpath('//title[starts-with(., "The")]/text()')
print(titulos_com_the)


["Harry Potter and the Philosopher's Stone", 'The Hobbit']
['NZD', 'GBP', 'BRL']
['19.99', '15.99', '19.99']
['The Hobbit', 'The Godfather']


Parte 2 Acesse o arquivo HTML em anexo

Exercício 3:

Utilize (apenas) seletores de CSS com lxml para selecionar:

    Todos os nomes de países dos atletas;
    Todos os nomes de atletas que possuem a letra J (maiúscula);
    Todos os nomes de atletas dos EUA (USA) e da Rússia.


In [5]:

from lxml import html

# Carregar o HTML diretamente da string ou arquivo
with open('..\data\TP3.html', 'r') as file:
    content = file.read()

# Parsear o conteúdo HTML e garantir que estamos usando _Element
root = html.fromstring(content)

# 1. Selecionar os nomes dos países dos atletas
paises = root.cssselect('p.country')
nomes_paises = [pais.text for pais in paises]
print("Países:", nomes_paises)

# 2. Selecionar os nomes de atletas que possuem a letra "J" maiúscula
atletas_com_j = root.cssselect('h2.name')
atletas_com_j_nome = [atleta.text for atleta in atletas_com_j if 'J' in atleta.text]
print("Atletas com J:", atletas_com_j_nome)

# 3. Selecionar os nomes de atletas dos EUA (USA) e da Rússia
atletas_usa_russia = root.cssselect('div.athlete')
atletas_dos_usa_russia = [
    atleta.cssselect('h2.name')[0].text 
    for atleta in atletas_usa_russia 
    if atleta.cssselect('p.country')[0].text in ['USA', 'Russia']
]
print("Atletas dos EUA e Rússia:", atletas_dos_usa_russia)




Países: ['USA', 'Canada', 'China', 'Mexico', 'Russia']
Atletas com J: ['John Doe', 'Jane Smith']
Atletas dos EUA e Rússia: ['John Doe', 'Olga Ivanova']


<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gabri\AppData\Local\Temp\ipykernel_12048\2872179499.py:4: SyntaxWarning: invalid escape sequence '\d'
  with open('..\data\TP3.html', 'r') as file:


Exercício 4:

Utilize seletores de CSS com BeautifulSoup e algum tratamento em Python que seja necessário para alocar as informações de cada atleta do arquivo HTML em um dataframe do Pandas a ser salvo em arquivo CSV e então responda:

    A média de idades dos atletas;
    O atleta ganhador do Speed Test;
    O atleta ganhador do Endurance Test. 

In [6]:
from bs4 import BeautifulSoup
import pandas as pd

# Carregar o arquivo HTML
with open('..\data\TP3.html', 'r', encoding='utf-8') as file:
    content = file.read()

# Parsear o conteúdo HTML com BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Encontrar todas as divs de atletas
atletas = soup.select('div.athlete')

# Listas para armazenar as informações dos atletas
dados = []

# Extraindo os dados de cada atleta
for atleta in atletas:
    nome = atleta.select_one('h2.name').text
    pais = atleta.select_one('p.country').text
    idade = int(atleta.select_one('p.age').text.split()[-1])
    rank = int(atleta.select_one('span.rank').text)
    
    speed_test = int(atleta.select('span.result')[0].text)
    endurance_test = int(atleta.select('span.result')[1].text)
    
    # Armazenar os dados em uma lista
    dados.append({
        'Nome': nome,
        'País': pais,
        'Idade': idade,
        'Rank': rank,
        'Speed Test': speed_test,
        'Endurance Test': endurance_test
    })

# Criar um DataFrame com os dados
df = pd.DataFrame(dados)

# Salvar o DataFrame em um arquivo CSV
df.to_csv('atletas_performance.csv', index=False)

# 1. Calcular a média de idades dos atletas
media_idades = df['Idade'].mean()

# 2. Encontrar o ganhador do Speed Test (maior pontuação)
ganhador_speed_test = df.loc[df['Speed Test'].idxmax()]['Nome']

# 3. Encontrar o ganhador do Endurance Test (maior pontuação)
ganhador_endurance_test = df.loc[df['Endurance Test'].idxmax()]['Nome']

# Exibir os resultados
print(f"Média de idades dos atletas: {media_idades:.2f} anos")
print(f"Ganhador do Speed Test: {ganhador_speed_test}")
print(f"Ganhador do Endurance Test: {ganhador_endurance_test}")


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gabri\AppData\Local\Temp\ipykernel_12048\797789504.py:5: SyntaxWarning: invalid escape sequence '\d'
  with open('..\data\TP3.html', 'r', encoding='utf-8') as file:


Média de idades dos atletas: 26.00 anos
Ganhador do Speed Test: John Doe
Ganhador do Endurance Test: Jane Smith


Parte 3 WebScraping 

Exercício 5: 

Acesse a URL https://www.scrapethissite.com/pages/simple/ e obtenha os dados de todos os países, salvando em um dataframe (sem utilizar o comando `pd.read_html`) com as seguintes colunas:

    Country;
    Capital;
    Population;
    Area.

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.scrapethissite.com/pages/simple/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


# Inicializando listas para armazenar os dados
countries = []
capitals = []
populations = []
areas = []

# Encontrar todas as divs de países
countries_divs = soup.select('div.col-md-4.country')

# Extraindo os dados de cada país
for country_div in countries_divs:
    # Nome do país
    country_name = country_div.select_one('h3.country-name').text.strip()
    
    # Capital do país
    country_capital = country_div.select_one('span.country-capital').text.strip()
    
    # População
    country_population = int(country_div.select_one('span.country-population').text.strip())
    
    # Área
    country_area = float(country_div.select_one('span.country-area').text.strip())
    
    # Adicionando os dados às listas
    countries.append(country_name)
    capitals.append(country_capital)
    populations.append(country_population)
    areas.append(country_area)

# Criando um DataFrame com os dados extraídos
df = pd.DataFrame({
    'Country': countries,
    'Capital': capitals,
    'Population': populations,
    'Area (km^2)': areas
})

# Salvando o DataFrame em um arquivo CSV
df.to_csv('countries_info.csv', index=False)

# Exibindo o DataFrame
print(df)

                  Country           Capital  Population  Area (km^2)
0                 Andorra  Andorra la Vella       84000        468.0
1    United Arab Emirates         Abu Dhabi     4975593      82880.0
2             Afghanistan             Kabul    29121286     647500.0
3     Antigua and Barbuda        St. John's       86754        443.0
4                Anguilla        The Valley       13254        102.0
..                    ...               ...         ...          ...
245                 Yemen             Sanaa    23495361     527970.0
246               Mayotte         Mamoudzou      159042        374.0
247          South Africa          Pretoria    49000000    1219912.0
248                Zambia            Lusaka    13460305     752614.0
249              Zimbabwe            Harare    11651858     390580.0

[250 rows x 4 columns]


Exercício 6: 

Escolha uma notícia qualquer do site https://www.romanews.com.br/ e obtenha o HTML da página via requisição utilizando a biblioteca que preferir (`urllib`, `requests`, etc.), salvando o arquivo no seu diretório. Então, a partir do arquivo HTML da notícia salva, construa códigos em Python utilizando `BeautifulSoup` para obter e salvar em um arquivo JSON, junto à URL da notícia e ao datetime do momento da requisição da página (os objetos datetime devem ter a informação da timezone e ser colocados em isoformat no momento de gerar o JSON):

    O objeto datetime da data e hora da publicação da notícia (dica: é sempre bom checar as tags `meta` dentro do `head` do arquivo.);
    O título da notícia;
    O corpo do texto da notícia;
    O subtítulo da notícia (se houver);
    O autor ou autores da notícia (se houver).

In [15]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pytz
import json

HEADER = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0'}
# URL da notícia
url = "https://www.romanews.com.br/brasil/infogripe-aponta-que-a-covid-19-se-espalha-por-cinco-estados-0924"

# Fazendo a requisição HTTP para obter o HTML da página
response = requests.get(url, headers=HEADER, timeout=30)
html_content = response.text

# Salvando o HTML da página localmente
with open('noticia.html', 'w', encoding='utf-8') as file:
    file.write(html_content)


In [17]:
# Carregar o HTML salvo para o BeautifulSoup
with open('noticia.html', 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

# Título da notícia
titulo = soup.find('h1').text.strip()

# Subtítulo da notícia (se houver)
subtitulo_tag = soup.find('h2')
subtitulo = subtitulo_tag.text.strip() if subtitulo_tag else None

# Corpo da notícia
paragrafos = soup.find_all('p')
corpo = ' '.join([p.text for p in paragrafos])

# Autor(es) da notícia (se houver)
autor_tag = soup.find(class_='author-name')
autor = autor_tag.text.strip() if autor_tag else None

# Data e hora da publicação (procurando nas meta tags do head)
data_publicacao_tag = soup.find('meta', {'property': 'article:published_time'})
data_publicacao = data_publicacao_tag['content'] if data_publicacao_tag else None

# Data e hora atual no momento da requisição (com timezone)
data_atual = datetime.now(pytz.timezone('America/Sao_Paulo')).isoformat()

# Criando um dicionário para armazenar os dados
dados_noticia = {
    'url': url,
    'titulo': titulo,
    'subtitulo': subtitulo,
    'autor': autor,
    'data_publicacao': data_publicacao,
    'data_requisicao': data_atual,
    'corpo': corpo
}

# Salvando os dados em um arquivo JSON
with open('noticia.json', 'w', encoding='utf-8') as json_file:
    json.dump(dados_noticia, json_file, ensure_ascii=False, indent=4)

print("Dados da notícia salvos em noticia.json")


Dados da notícia salvos em noticia.json


Exercício 7: 

Repita a questão anterior utilizando um site de notícias de sua escolha.

In [18]:
url =  'https://www.uol.com.br/esporte/futebol/ultimas-noticias/2024/09/14/corinthians-ve-holofotes-em-depay-e-se-empolga-com-reforco-coadjuvante.htm'
# Fazendo a requisição HTTP para obter o HTML da página
response = requests.get(url, headers=HEADER, timeout=30)
html_content = response.text

# Salvando o HTML da página localmente
with open('noticia_uol.html', 'w', encoding='utf-8') as file:
    file.write(html_content)

In [20]:
# Carregar o HTML salvo para o BeautifulSoup
with open('noticia_uol.html', 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

# Título da notícia
titulo = soup.find('h1').text.strip()

# Subtítulo da notícia (se houver)
subtitulo_tag = soup.find('h2')
subtitulo = subtitulo_tag.text.strip() if subtitulo_tag else None

# Corpo da notícia
paragrafos = soup.find_all('p')
corpo = ' '.join([p.text for p in paragrafos])

# Autor(es) da notícia (se houver)
autor_tag = soup.find(class_='solar-author-name')
autor = autor_tag.text.strip() if autor_tag else None

# Procurar pela tag <time> com a classe 'date'
time_tag = soup.find('time', class_='date')
data_publicacao = time_tag['datetime']

# Data e hora atual no momento da requisição (com timezone)
data_atual = datetime.now(pytz.timezone('America/Sao_Paulo')).isoformat()

# Criando um dicionário para armazenar os dados
dados_noticia = {
    'url': url,
    'titulo': titulo,
    'subtitulo': subtitulo,
    'autor': autor,
    'data_publicacao': data_publicacao,
    'data_requisicao': data_atual,
    'corpo': corpo
}

# Salvando os dados em um arquivo JSON
with open('noticia_uol.json', 'w', encoding='utf-8') as json_file:
    json.dump(dados_noticia, json_file, ensure_ascii=False, indent=4)

print("Dados da notícia salvos em noticia_uol.json")

Dados da notícia salvos em noticia_uol.json


Parte 4 WebCrawling

Exercício 8: 

Examine o site https://difusoranews.com/ e descubra o padrão de URL para paginação que ele aceita (dica: considere tentativa-e-erro). Então, utilize-o para obter uma lista de links de notícias requisitando as cinco primeiras páginas e raspando os links através de um único seletor de CSS aplicado via `BeautifulSoup`.

In [4]:
import requests
from bs4 import BeautifulSoup

# Lista das categorias
categorias = [
    'bem-estar', 'cultura', 'economia', 'educacao', 'entretenimento', 
    'esportes', 'maranhao', 'mundo', 'policia', 'politica', 'oportunidade', 'tecnologia'
]
HEADER = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0'}
# URL base do site
url_base = "https://difusoranews.com/{}/page/{}/"

# Set para armazenar os links únicos
links_unicos = set()

# Loop sobre as categorias e páginas
for categoria in categorias:
    for pagina in range(1, 6):  # Pegar as 5 primeiras páginas
        url = url_base.format(categoria, pagina)
        response = requests.get(url, headers=HEADER, timeout=30)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Selecionar todos os links de notícias pela tag <a> e a classe correta
            noticias = soup.select('h3 a')
            
            for noticia in noticias:
                link = noticia['href']
                links_unicos.add(link)
        else:
            print(f"Erro ao acessar a página: {url}")

# Exibir os links únicos
for link in links_unicos:
    print(link)

# Salvar os links únicos em um arquivo de texto
with open('links_noticias.txt', 'w') as f:
    for link in links_unicos:
        f.write(link + '\n')


https://difusoranews.com/bem-estar/um-a-cada-tres-brasileiros-nao-sabe-que-e-hipertenso/
https://difusoranews.com/esportes/maranhense-rayane-soares-da-show-na-pista-com-ouro-e-recorde-mundial-em-paris/
https://difusoranews.com/policia/video-dupla-e-presa-tentando-assaltar-farmacia-em-sao-luis/
https://difusoranews.com/esportes/sampaio-enfrenta-abc-em-confronto-direto-pela-permanencia-na-serie-c/
https://difusoranews.com/politica/saidinha-temporaria-veja-como-votaram-os-deputados-maranhenses/
https://difusoranews.com/bem-estar/ondas-de-calor-da-menopausa-sao-mais-perigosas-do-que-se-pensava-dizem-estudos-2/
https://difusoranews.com/maranhao/eclipse-lunar-podera-ser-visto-no-maranhao-nesta-terca-feira-17/
https://difusoranews.com/entretenimento/davi-vence-prova-e-esta-na-final-do-bbb-2024/
https://difusoranews.com/mundo/jogador-de-futebol-e-assassinado-a-tiros-durante-velorio-no-equador/
https://difusoranews.com/policia/homem-disfarcado-de-policial-assalta-agencia-bancaria-em-pinheiro/
h

Exercício 9:

Repita a questão anterior utilizando um site de notícias de sua escolha.


In [5]:
categorias = [
    'cz74k717pw5t', 'c30gn378n6kt', 'cmdm4ynm24kt', 'cvjp2jr0k9rt', 'c340q430z4vt', 
    'cr50y580rjxt', 'c404v027pd4t', 'c9y2j35dn2zt', 'cxndrr1qgllt'
]
HEADER = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0'}
# URL base do site
url_base = "https://www.bbc.com/portuguese/topics/{}?page={}"

# Set para armazenar os links únicos
links_unicos = set()

# Loop sobre as categorias e páginas
for categoria in categorias:
    for pagina in range(1, 6):  # Pegar as 5 primeiras páginas
        url = url_base.format(categoria, pagina)
        response = requests.get(url, headers=HEADER, timeout=30)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Selecionar todos os links de notícias pela tag <a> e a classe correta
            noticias = soup.select('h2 a')
            
            for noticia in noticias:
                link = noticia['href']
                links_unicos.add(link)
        else:
            print(f"Erro ao acessar a página: {url}")

# Exibir os links únicos
for link in links_unicos:
    print(link)

# Salvar os links únicos em um arquivo de texto
with open('links_noticias_bbc.txt', 'w') as f:
    for link in links_unicos:
        f.write(link + '\n')

https://www.bbc.com/portuguese/articles/ce38jyjjn7vo
https://www.bbc.com/portuguese/articles/c4gll1yrjx1o
https://www.bbc.com/portuguese/articles/c0d324vnv21o
https://www.bbc.com/portuguese/articles/cx2n3ylq60xo
https://www.bbc.com/portuguese/articles/czq44g6z082o
https://www.bbc.com/portuguese/articles/cj7d7kej8zno
https://www.bbc.com/portuguese/articles/c0jq130yl5ko
https://www.bbc.com/portuguese/articles/c3vx7ygkeqro
https://www.bbc.com/portuguese/articles/c3gg3e8nyero
https://www.bbc.com/portuguese/articles/cv2j24d26gyo
https://www.bbc.com/portuguese/articles/cj62l5kj5x4o
https://www.bbc.com/portuguese/articles/c623gedpy1eo
https://www.bbc.com/portuguese/articles/ce8vxg1xgx3o
https://www.bbc.com/portuguese/articles/cl7l79vyyyko
https://www.bbc.com/portuguese/articles/cld4kr6119vo
https://www.bbc.com/portuguese/articles/c8495wknqwvo
https://www.bbc.com/portuguese/articles/cd10zp4r7yjo
https://www.bbc.com/portuguese/articles/cn0lx86y29do
https://www.bbc.com/portuguese/articles/c1jlxy

Exercício 10: 

Consulte o arquivo `robots.txt` do site https://tecnoblog.net/ para encontrar o link do `sitemap_index` e obtenha a partir dele a lista de todos os sitemaps de notícias (filtrando apenas os de notícias).


In [9]:

HEADER = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0'}
# URL do sitemap principal
url = 'https://tecnoblog.net/sitemap_index.xml'
links = []
# Faz a requisição do conteúdo do sitemap
response = requests.get(url, headers=HEADER, timeout=30)

# Analisa o conteúdo XML com BeautifulSoup
soup = BeautifulSoup(response.content, 'xml')

# Extrai e imprime todos os links que contêm "noticias"
for sitemap in soup.find_all('loc'):
    link = sitemap.text
    if 'noticias' in link:
        print(link)
        links.append(link)
        
        
# Salvar os links únicos em um arquivo de texto
with open('noticias_sitemap_tecnoblog.txt', 'w') as f:
    for link in links:
        f.write(link + '\n')



https://tecnoblog.net/noticias-sitemap.xml
https://tecnoblog.net/noticias-sitemap2.xml
https://tecnoblog.net/noticias-sitemap3.xml
https://tecnoblog.net/noticias-sitemap4.xml
https://tecnoblog.net/noticias-sitemap5.xml
https://tecnoblog.net/noticias-sitemap6.xml
https://tecnoblog.net/noticias-sitemap7.xml
https://tecnoblog.net/noticias-sitemap8.xml
https://tecnoblog.net/noticias-sitemap9.xml
https://tecnoblog.net/noticias-sitemap10.xml
https://tecnoblog.net/noticias-sitemap11.xml
https://tecnoblog.net/noticias-sitemap12.xml
https://tecnoblog.net/noticias-sitemap13.xml
https://tecnoblog.net/noticias-sitemap14.xml
https://tecnoblog.net/noticias-sitemap15.xml
https://tecnoblog.net/noticias-sitemap16.xml
https://tecnoblog.net/noticias-sitemap17.xml
https://tecnoblog.net/noticias-sitemap18.xml
https://tecnoblog.net/noticias-sitemap19.xml
https://tecnoblog.net/noticias-sitemap20.xml
https://tecnoblog.net/noticias-sitemap21.xml
https://tecnoblog.net/noticias-sitemap22.xml
https://tecnoblog.ne

Parte 5 Scrapy

Exercício 11: 

Escolha um dos sites de exemplo em https://toscrape.com/ para criar um projeto no Scrapy que obtenha os principais dados disponíveis, salvando em CSV ou JSON conforme a conveniência em relação aos dados do site escolhido.


Exercício 12: 

Escolha um dos sites utilizados nas questões 6, 7, 8 e 9 para montar um projeto no Scrapy que abarque tanto o Crawling quanto o Scraping, a fim de rodá-lo tal como na questão anterior.